In [10]:
import pandas as pd
import glob

def combine_eGFP_and_percentage_data(project_files, output_filename="combined_results.xlsx"):
    """
    Combines data from multiple Excel files, each containing 'Classification', 'Average eGFP',
    and 'Percentage of Total' columns.  The output displays these values for each file,
    along with the average and standard deviation for both 'Average eGFP' and
    'Percentage of Total' across all files.  This version avoids using pivot().

    Args:
        project_files (list): A list of paths to the Excel files.
        output_filename (str, optional): The name of the Excel file to save the results to.
                                         Defaults to "combined_results.xlsx".
    """
    all_project_data = []
    for file in project_files:
        try:
            print(f"Processing file: {file}")
            df = pd.read_excel(file)
            print(f"Raw data from file '{file}':\n{df.head()}")  # Print the first few rows

            # Check for required columns
            if "Classification" in df.columns and "Average eGFP" in df.columns and "Percentage of Total" in df.columns:
                # Extract the filename (without extension)
                project_name = file.split('/')[-1].split('\\')[-1].split('.')[0]
                df = df.fillna(0)  # fill NA with 0
                df['File'] = project_name #add file identifier
                all_project_data.append(df)
            else:
                print(
                    f"Warning: File '{file}' does not contain required 'Classification', 'Average eGFP', and 'Percentage of Total' columns. Skipping.")
                continue  # Skip to the next file
        except FileNotFoundError:
            print(f"Error: File '{file}' not found.")
        except Exception as e:
            print(f"Error reading file '{file}': {e}")

    if not all_project_data:
        print("No valid project data found in the provided files.")
        return

    # Combine all project data into a single DataFrame
    combined_df = pd.concat(all_project_data, ignore_index=True)

    # Calculate averages and standard deviations
    def agg_func(x):
        return pd.Series({
            'Average eGFP': x['Average eGFP'].mean(),
            'Stdev eGFP': x['Average eGFP'].std(),
            'Average Percentage': x['Percentage of Total'].mean(),
            'Stdev Percentage': x['Percentage of Total'].std(),
        })

    # Group by 'Classification'
    grouped_df = combined_df.groupby('Classification').apply(agg_func)

    # Unstack the results
    unstacked_df = combined_df.set_index(['Classification', 'File']).unstack()
    unstacked_df.columns = [f"{file}_{stat}" for file, stat in unstacked_df.columns]

    # Join the grouped data
    final_df = grouped_df.join(unstacked_df, how='outer')
    final_df = final_df.reset_index()

    # Sort by 'Average eGFP' in descending order
    if "Average eGFP" in final_df.columns:
        final_df = final_df.sort_values(by="Average eGFP", ascending=False)

    try:
        final_df.to_excel(output_filename, index=False)
        print(f"\nCombined eGFP and Percentage data saved to '{output_filename}'")
    except Exception as e:
        print(f"Error saving results to '{output_filename}': {e}")



if __name__ == "__main__":
    # Look for any .xlsx files in the current directory
    project_files = glob.glob("*_results.xlsx")

    if not project_files:
        print("No .xlsx files found in the current directory. Please ensure the output files from the first script are in the same directory.")
    else:
        print(f"Found the following .xlsx files: {project_files}")
        combine_eGFP_and_percentage_data(project_files)


Found the following .xlsx files: ['average_eGFP5_results.xlsx', 'average_eGFP16FH_results.xlsx', 'average_eGFP21_results.xlsx', 'average_eGFP6_results.xlsx']
Processing file: average_eGFP5_results.xlsx
Raw data from file 'average_eGFP5_results.xlsx':
  Classification  Average eGFP  Percentage of Total
0     Right: TTd   9156.006053             8.120731
1    Right: root   8576.125386             7.606418
2      Right: TT   7961.423528             7.061221
3    Right: grey   6486.373953             5.752956
4      Right: CH   6266.372500             5.557830
Processing file: average_eGFP16FH_results.xlsx
Raw data from file 'average_eGFP16FH_results.xlsx':
  Classification  Average eGFP  Percentage of Total
0   Right: ORBm1  11365.697450             6.811656
1    Right: root   9453.824860             5.665838
2     Right: ORB   9001.826514             5.394948
3    Right: grey   8589.459482             5.147809
4      Right: CH   6571.747644             3.938560
Processing file: average_e

/tmp/ipykernel_163245/1511333715.py:56: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped_df = combined_df.groupby('Classification').apply(agg_func)



Combined eGFP and Percentage data saved to 'combined_results.xlsx'
